# Hailo HEF Export (Raspberry PI AI Hat 13 TOPS)

This export notebook it to help with converting a yolo pytorch model to .hef format for use on a raspberry pi ai hat 13 tops model.

Under the hood this is a hailo-8l architecture.

There are 2 examples, one for a yolo object detection model, one for a segmentation model.

Machine setup:

This notebook has been tested on a 4060 TI 16GB x86 machine with 32 Gb RAM running Ubuntu 22.04

This notebook does *NOT* run in google colab free tier, it might work on some of the larger paid hardware options however.

Start by copying uploading your pytorch weights from the yolo training process.

We will also pull the dataset from oslo.vision and use that for the calibration images. If you are not using oslo just upload a folder of images.


In [ ]:
!pip3 install -U ultralytics

import ultralytics as ul
ul.checks()

import os

HOME = os.getcwd()
print("HOME:", HOME)

In [ ]:
# install hailo deps

!wget https://s3.amazonaws.com/excitedmedia/hailo_dataflow_compiler-3.32.0-py3-none-linux_x86_64.whl
!git clone https://github.com/hailo-ai/hailo_model_zoo.git
!sudo apt update && sudo apt install -y graphviz graphviz-dev
!python3 -m venv hailo
!source hailo/bin/activate
!pip install hailo_dataflow_compiler-3.32.0-py3-none-linux_x86_64.whl
!cd hailo_model_zoo && git checkout -b update-to-version-332-tag
!pip install -e .



In [ ]:
# pull Dataset from Oslo for use as calibration images in the convertion process.
# This assumes that you have your dataset in oslo.vision and have created an export in the correct format.
# (YOLO for Object Detection, YOLO-Seg for segmentation)

!pip3 install -U oslovision

from oslovision import OsloVision

project_identifier = "[inserthere]"
access_token = "[inserthere]"
dataset_version = "[inserthere]"

oslo = OsloVision(access_token)

# Download dataset
dataset_path = oslo.download_export(project_identifier, dataset_version, HOME)

print("Dataset path:", dataset_path)

In [ ]:
# export to onnx format

!yolo export model={HOME}/best.pt imgsz=640 format=onnx opset=11

In [ ]:
# to export a yolov8n object detection model
ONNX_MODEL = f"{HOME}/best.onnx"
CALIBRATION_IMAGES = f"{dataset_path}/train"
# this num_classes should represent the number of classes in the model
NUM_CLASSES = 3

!hailomz compile yolov8n --ckpt={ONNX_MODEL} --hw-arch hailo8l --calib-path {CALIBRATION_IMAGES} --classes {NUM_CLASSES}



In [ ]:
# to export a yolov8s_seg segmentation model

ONNX_MODEL = f"{HOME}/best.onnx"
CALIBRATION_IMAGES = f"{dataset_path}/train"
NUM_CLASSES = 1

!hailomz compile yolov8s_seg --ckpt={ONNX_MODEL} --hw-arch hailo8l --calib-path {CALIBRATION_IMAGES} --classes {NUM_CLASSES} --performance